# Determining Within Z Dimension Stability with SVCCA

**Gregory Way 2018**

Here, we apply Singular Vector Canonical Correlation Analysis ([Raghu et al. 2017](https://arxiv.org/abs/1706.05806 "SVCCA: Singular Vector Canonical Correlation Analysis for Deep Learning Dynamics and Interpretability")) ([github](https://github.com/google/svcca)) to the neuron matrices Z to quantify model stability both within and across algorithms over each bottleneck dimensionality.

Briefly, SVCCA uses Singular Value Decomposition (SVD) to extract the components explaining 99% of the variation.
This is done to remove potential dimensions described by noise.
Next, SVCCA performs a Canonical Correlation Analysis (CCA) on the SVD matrices to identify maximum correlations of linear combinations of both input matrices. The algorithm will identify the canonical correlations of highest magnitude across and within algorithms of the same dimensionality. 

The output of the SVCCA analysis is the SVCCA mean similarity score. This single number can be interpreted as a measure of similarity, or stability, in the solutions identified from each compression algorithm.

In [1]:
import os
import glob
import numpy as np
import pandas as pd

from scripts.util import read_in_z, get_svcca_across_algorithm_stability

In [2]:
datasets = ['TARGET', 'TCGA', 'GTEX']
algorithms = ['pca', 'ica', 'nmf', 'dae', 'vae']
z_dims = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30,
          35, 40, 45, 50, 60, 70, 80, 90, 100, 125, 150, 200]
shuffled_data = (True, False)

In [3]:
large_svcca_results_list = []
for dataset in datasets:

    for z in z_dims:

        for signal in shuffled_data:

            if signal:
                shuffled_status = 'shuffled'
            else:
                shuffled_status = 'signal'

            print("Calculating... dataset {} for {} dimension {}"
                  .format(dataset, shuffled_status, z))

            # Read in the z matrix of interest
            z_dict = read_in_z(
                dataset=dataset,
                z_dim=z,
                algorithm='all',
                shuffled_data=signal
            )

            # Perform across algorithm SVCCA
            svcca_out = (
                get_svcca_across_algorithm_stability(z_dict=z_dict,
                                                     algorithms=algorithms)
            )
            
            # Append info to the output dataframe
            svcca_out = svcca_out.assign(
                dataset=dataset,
                z_dim=z,
                shuffled=shuffled_status
            )

            large_svcca_results_list.append(svcca_out)

Calculating... dataset TARGET for shuffled dimension 2


/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:285: RuntimeWarning: invalid value encountered in true_divide
  sigmayy /= ymax
/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:286: RuntimeWarning: invalid value encountered in true_divide
  sigmaxy /= np.sqrt(xmax * ymax)
/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:287: RuntimeWarning: invalid value encountered in true_divide
  sigmayx /= np.sqrt(xmax * ymax)
/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:83: RuntimeWarning: invalid value encountered in greater_equal
  y_idxs = (y_diag >= threshold)
/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:284: RuntimeWarning: invalid value encountered in true_divide
  sigmaxx /= xmax
/home/gway/repos/interpret-compression/4.analyze-components/scripts/cca_core.py:82: RuntimeWarning: invalid value encountered in greater_equal
  x_idxs 

Calculating... dataset TARGET for signal dimension 2
Calculating... dataset TARGET for shuffled dimension 3
Calculating... dataset TARGET for signal dimension 3
Calculating... dataset TARGET for shuffled dimension 4
Calculating... dataset TARGET for signal dimension 4
Calculating... dataset TARGET for shuffled dimension 5
Calculating... dataset TARGET for signal dimension 5
Calculating... dataset TARGET for shuffled dimension 6
Calculating... dataset TARGET for signal dimension 6
Calculating... dataset TARGET for shuffled dimension 7
Calculating... dataset TARGET for signal dimension 7
Calculating... dataset TARGET for shuffled dimension 8
Calculating... dataset TARGET for signal dimension 8
Calculating... dataset TARGET for shuffled dimension 9
Calculating... dataset TARGET for signal dimension 9
Calculating... dataset TARGET for shuffled dimension 10
Calculating... dataset TARGET for signal dimension 10
Calculating... dataset TARGET for shuffled dimension 12
Calculating... dataset TA

Calculating... dataset GTEX for signal dimension 70
Calculating... dataset GTEX for shuffled dimension 80
Calculating... dataset GTEX for signal dimension 80
Calculating... dataset GTEX for shuffled dimension 90
Calculating... dataset GTEX for signal dimension 90
Calculating... dataset GTEX for shuffled dimension 100
Calculating... dataset GTEX for signal dimension 100
Calculating... dataset GTEX for shuffled dimension 125
Calculating... dataset GTEX for signal dimension 125
Calculating... dataset GTEX for shuffled dimension 150
Calculating... dataset GTEX for signal dimension 150
Calculating... dataset GTEX for shuffled dimension 200
Calculating... dataset GTEX for signal dimension 200


In [4]:
svcca_results_df = pd.concat(large_svcca_results_list)
svcca_results_df.columns = ['seed_1', 'seed_2', 'algorithm_1', 'algorithm_2',
                            'svcca_mean_similarity', 'dataset', 'z_dim', 'shuffled']
svcca_results_df.head()

,seed_1,seed_2,algorithm_1,algorithm_2,svcca_mean_similarity,dataset,z_dim,shuffled
0,242178,320346,pca,pca,0.546753,TARGET,2,shuffled
1,242178,320346,pca,ica,0.548949,TARGET,2,shuffled
2,242178,320346,pca,nmf,0.575693,TARGET,2,shuffled
3,242178,320346,pca,dae,0.000000,TARGET,2,shuffled
4,242178,320346,pca,vae,0.513674,TARGET,2,shuffled


In [5]:
out_file = os.path.join('results', 'svcca_mean_correlation_within_z.tsv.gz')
svcca_results_df.to_csv(out_file, sep='\t', index=False, compression='gzip')